In [19]:
import pandas as pd
import datetime as dt
import altair as alt
import re
import datetime
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [20]:
bicing_stations=pd.read_csv("bicing_station_districts.csv")

september=pd.read_csv("../2020_09_Setembre_BicingNou_ESTACIONS.csv")

# Loading and cleaning data from Bicing Station's activities

First, we want to get the day of September in which each row was recorded, using the data in column "last_updated" and with the function fromtimestamp in the datetime library.

In [3]:
september["day"]=september["last_updated"].transform(lambda x : int(dt.datetime.fromtimestamp(x).strftime("%d")))

Next we will compute some meaningful variables that we will use later on:

In [4]:
september["Occupation"] = september["num_bikes_available"]/(september["num_docks_available"]+september["num_bikes_available"])

In [5]:
september.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl,day,Occupation
0,1,19,19,0,23,1,1,1,1.598911e+09,True,IN_SERVICE,1598911239,28,1,0.452381
1,2,1,1,0,23,1,1,1,1.598911e+09,True,IN_SERVICE,1598911239,28,1,0.041667
2,3,4,4,0,21,1,1,1,1.598911e+09,True,IN_SERVICE,1598911239,28,1,0.160000
3,4,3,3,0,14,1,1,1,1.598911e+09,True,IN_SERVICE,1598911239,28,1,0.176471
4,5,3,3,0,35,1,1,1,1.598911e+09,True,IN_SERVICE,1598911239,28,1,0.078947


As we will be plotting the daily data about the activity of each station, we will group all rows by the day and the station and average the different metrics we will be plotting:

In [6]:
aggregate=september[september["status"]=="IN_SERVICE"].groupby(["day", "station_id"]).mean()
aggregate=aggregate.reset_index()

In [7]:
aggregate.columns=['Day', 'Station','avg_Bikes_Avail', 'avg_Mechanical_Bikes',
       'avg_Electric_Bikes', 'avg_Docks_Available',
       'is_installed', 'is_renting', 'is_returning', 'last_reported',
       'is_charging_station', 'last_updated', 'ttl', 'avg_Occupation']
aggregate=aggregate[['Day', 'Station','avg_Bikes_Avail', 'avg_Mechanical_Bikes',
       'avg_Electric_Bikes', 'avg_Docks_Available', 'avg_Occupation']]

We will create a variable "Date", containing the date in a format that Altair will understand as so:

In [8]:
aggregate["Date"]=aggregate['Day'].apply(lambda x: datetime.datetime(2020, 9, x))

We will also create a "Day_of_week" variable, a string containing the name of the days in plain text:

In [9]:
aggregate["Day_of_week"]=aggregate['Day'].apply(lambda x: datetime.datetime.strftime(datetime.datetime(2020, 9, x), '%A'))

In [10]:
aggregate

,Day,Station,avg_Bikes_Avail,avg_Mechanical_Bikes,avg_Electric_Bikes,avg_Docks_Available,avg_Occupation,Date,Day_of_week
0,1,1,13.722222,13.552083,0.170139,27.763889,0.328943,2020-09-01,Tuesday
1,1,2,3.093750,2.982639,0.111111,20.569444,0.131970,2020-09-01,Tuesday
2,1,3,6.593750,6.461806,0.131944,16.878472,0.282516,2020-09-01,Tuesday
3,1,4,5.909722,5.645833,0.263889,10.642361,0.360665,2020-09-01,Tuesday
4,1,5,6.277778,6.125000,0.152778,32.336806,0.161981,2020-09-01,Tuesday
...,...,...,...,...,...,...,...,...,...
14746,30,515,8.371528,8.336806,0.034722,15.628472,0.348814,2020-09-30,Wednesday
14747,30,516,14.072917,13.513889,0.559028,6.927083,0.670139,2020-09-30,Wednesday
14748,30,517,4.715278,4.010417,0.704861,18.284722,0.205012,2020-09-30,Wednesday
14749,30,518,0.472222,0.149306,0.322917,25.527778,0.018162,2020-09-30,Wednesday


# Loading data about the Bicing Stations

Now, we will take a look at the dataframe with the information about the location of the stations 

In [11]:
bicing_stations.head(3)

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,Latlon,Barri,Districte,Municipi
0,36,"AV. DE LA CATEDRAL, 6",ELECTRICBIKESTATION,41.385062,2.176683,8,"AV. DE LA CATEDRAL, 6",8002,21,"41.38506160000001,2.1766834",el Gòtic,Ciutat Vella,Barcelona
1,53,PL. CARLES PI I SUNYER,ELECTRICBIKESTATION,41.385086,2.174016,10,PL. CARLES PI I SUNYER,8002,21,"41.385086,2.174016",el Gòtic,Ciutat Vella,Barcelona
2,55,"LA RAMBLA, 80",ELECTRICBIKESTATION,41.381428,2.173286,8,"LA RAMBLA, 80",8002,19,"41.3814279,2.1732861",el Gòtic,Ciutat Vella,Barcelona


Let us check if we don't have data from some of the stations we want to analyse:

In [12]:
no_info = []
for station in set(aggregate["Station"]):
    if int(station) not in list(bicing_stations["station_id"]):
        no_info.append(int(station))
print("No info of stations:", no_info)

No info of stations: [47, 231, 330, 399, 430, 449, 456, 500, 513, 529]


We will discard the rows from our data set that correspond to one of these Bicing Stations, as we don't have the information corresponding to their location. 

In [13]:
aggregate=aggregate[~aggregate.Station.isin(no_info)]


We will also discard rows from the Bicing Station data that don't correspond to stations in Barcelona (i.e. L'Hospitalet de Llobregat, Sant Adrià del Besós...)

In [14]:
bicing_stations_filtered=bicing_stations[bicing_stations["Municipi"]=="Barcelona"]
bicing_stations_filtered=bicing_stations_filtered[["station_id","name", "lat", "lon", "altitude","Barri", "Districte"]]
bicing_stations_filtered.columns=["Station", "Name", "Lat", "Lon", "Altitude", "Barri", "Districte"]
bicing_stations_filtered.head()

,Station,Name,Lat,Lon,Altitude,Barri,Districte
0,36,"AV. DE LA CATEDRAL, 6",41.385062,2.176683,8,el Gòtic,Ciutat Vella
1,53,PL. CARLES PI I SUNYER,41.385086,2.174016,10,el Gòtic,Ciutat Vella
2,55,"LA RAMBLA, 80",41.381428,2.173286,8,el Gòtic,Ciutat Vella
3,57,"RAMBLA, 2",41.376876,2.177225,4,el Gòtic,Ciutat Vella
4,126,PG. DE COLOM /VIA LAIETANA,41.380628,2.182192,4,el Gòtic,Ciutat Vella


In this project we will be plotting the activity on the different districts using the geojson in https://raw.githubusercontent.com/martgnz/bcn-geodata/master/districtes/districtes.geojson, so we will change the names given to the districts in the csv to match the ones in the geojson file.

In [15]:
def changeNames(district):
    if district=="les Corts":
        return "Les Corts"
    elif district=="Sarrià - Sant Gervasi":
        return "Sarrià-Sant Gervasi"
    elif district=="Sant Antoni":
        return "Eixample"
    return district
    

bicing_stations_filtered["Districte"]=bicing_stations_filtered["Districte"].apply(lambda x: changeNames(x))

There are also some problems regarding the names of some neighbourhoods:

In [16]:
def changeBarris(barri):
    if re.search("Carrer.*", barri) or barri=="Avinguda Diagonal":
        return "les Corts"
    if barri=="Superilla del Poblenou":
        return "el Parc i la Llacuna del Poblenou"
    return barri


bicing_stations_filtered["Barri"]=bicing_stations_filtered["Barri"].apply(lambda x: changeBarris(x))

Next, we will join both datasets in order to be able to plot data using information in both of them:

In [17]:
Activity_Station_Day_Info=aggregate.set_index('Station').join(bicing_stations_filtered.set_index('Station'), how="inner").reset_index()
Activity_Station_Day_Info.head(3)

,Station,Day,avg_Bikes_Avail,avg_Mechanical_Bikes,avg_Electric_Bikes,avg_Docks_Available,avg_Occupation,Date,Day_of_week,Name,Lat,Lon,Altitude,Barri,Districte
0,1,1,13.722222,13.552083,0.170139,27.763889,0.328943,2020-09-01,Tuesday,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16,el Fort Pienc,Eixample
1,1,2,9.479167,9.291667,0.187500,32.395833,0.231612,2020-09-02,Wednesday,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16,el Fort Pienc,Eixample
2,1,3,8.642361,8.454861,0.187500,37.340278,0.187898,2020-09-03,Thursday,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16,el Fort Pienc,Eixample


Finally, we store the resulting data frame in a csv file (clean_data.csv) that will be used later on when designing the visualizations:

In [18]:
Activity_Station_Day_Info.to_csv("clean_data.csv")